In [1]:
from google.colab import drive
drive.mount('/content/dirve')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install gTTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#Voice Library
import numpy as np
import librosa
import random
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')
from pydub import AudioSegment
from time import sleep
import speech_recognition as sr


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(927)

from tensorflow.keras.models import load_model


# NLP Library
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel
from gtts import gTTS
from IPython.display import Audio 

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

# 모델 호출

In [7]:
CP2_PATH = '/content/dirve/MyDrive/CP2/'

In [8]:
mels_model = load_model(CP2_PATH+'Model/Voice/melspectrogram_model.h5')
mfcc_model = load_model(CP2_PATH+'Model/Voice/mfcc_model.h5')

In [9]:
ha_model = TFGPT2LMHeadModel.from_pretrained(CP2_PATH+'Model/Chat Bot/KoGPT-2_happy_model.h5')
sa_model = TFGPT2LMHeadModel.from_pretrained(CP2_PATH+'Model/Chat Bot/KoGPT-2_sad_model.h5')
an_model = TFGPT2LMHeadModel.from_pretrained(CP2_PATH+'Model/Chat Bot/KoGPT-2_angry_model.h5')
em_model = TFGPT2LMHeadModel.from_pretrained(CP2_PATH+'Model/Chat Bot/KoGPT-2_embarassment_model.h5')
ne_model = TFGPT2LMHeadModel.from_pretrained(CP2_PATH+'Model/Chat Bot/KoGPT-2_neutrality_model.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/dirve/MyDrive/CP2/Model/Chat Bot/KoGPT-2_happy_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/dirve/MyDrive/CP2/Model/Chat Bot/KoGPT-2_sad_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/dirve/MyDrive/CP2/Model/Chat Bot/KoGPT-2_angry_model.h5.
If your ta

In [10]:
# 사전학습 토크나이저 호출
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Voice Part

In [11]:
class Classification:
    CP2_PATH = '/content/dirve/MyDrive/CP2/'
    
    size = 120
    pad_size = 450
    sr = 16000
    
    def __init__(self, mels_model, mfcc_model):
        self.mels_model = mels_model
        self.mfcc_model = mfcc_model
    


    # 오디오 파일 형식 변환
    def transform_audio(self, AUDIO_PATH):
        if AUDIO_PATH[-3:] == 'wav':
            return AUDIO_PATH
        else:
            sound = AudioSegment.from_file(AUDIO_PATH)
            audio_name = AUDIO_PATH[AUDIO_PATH.rfind('/')+1:][:-3]
            NEW_AUDIO_PATH = self.CP2_PATH+'Person_voice/'+audio_name+'wav'
            sound.export(NEW_AUDIO_PATH, format='wav')
            return NEW_AUDIO_PATH


    # 오디오 불러오기
    def load_audio(self, AUDIO_PATH):
        self.AUDIO_PATH = AUDIO_PATH
        audio, _ = librosa.load(AUDIO_PATH, sr=self.sr)
        audio_array = np.array(audio)

        return audio

    # 패딩
    def random_pad(self, mels, pad_size, mfcc=True):
        pad_width = self.pad_size - mels.shape[1]
        rand = np.random.rand()
        left = int(pad_width * rand)
        right = pad_width - left

        if mfcc:
            mels = np.pad(mels, pad_width=((0,0), (left,right)), mode='constant')
            local_max, local_min = mels.max(), mels.min()
            mels = (mels - local_min)/(local_max - local_min)
        else:
            local_max, local_min = mels.max(), mels.min()
            mels = (mels - local_min)/(local_max - local_min)
            mels = np.pad(mels, pad_width=((0,0), (left,right)), mode='constant')

        return mels

    
    # melspectrogram, mfcc array
    def mels_mfcc_array(self, audio):
        self.audio = audio
        audio_mels = []
        audio_mfcc = []

        mels = librosa.feature.melspectrogram(self.audio, sr=self.sr, n_mels=self.size)
        mels = librosa.power_to_db(mels, ref=np.max)

        mfcc = librosa.feature.mfcc(self.audio, sr=self.sr, n_mfcc=self.size)

        audio_mels.append(Classification.random_pad(self, mels, pad_size=self.pad_size, mfcc=False))
        audio_mfcc.append(Classification.random_pad(self, mfcc, pad_size=self.pad_size, mfcc=True))

        audio_mels_array = np.array(audio_mels, np.float64)
        audio_mfcc_array = np.array(audio_mfcc, np.float64)

        audio_mels_array = np.expand_dims(audio_mels_array, -1)
        audio_mfcc_array = np.expand_dims(audio_mfcc_array, -1)

        print("audio mels shape: ", audio_mels_array.shape)
        print("audio mfcc shape: ", audio_mfcc_array.shape)


        return audio_mels_array, audio_mfcc_array


    # 예측
    def predict(self, mels_audio_array, mfcc_audio_array):
        preds_val_list = []
        
        mels_pred = self.mels_model.predict(mels_audio_array)
        preds_val_list.append(mels_pred)

        mfcc_pred = self.mfcc_model.predict(mfcc_audio_array)
        preds_val_list.append(mfcc_pred)

        pred_result = preds_val_list[0].copy()
        for i in range(1, len(preds_val_list)):
            pred_result += preds_val_list[i]
        pred_label = np.argmax(pred_result, axis=1)
        print(pred_result)
        
        return pred_label



In [12]:
def emotion_predict(AUDIO_PATH):
    emotion = Classification(mels_model, mfcc_model)
    audio_path = emotion.transform_audio(AUDIO_PATH)
    print('파일 형식 변환 중....')
    sleep(7)
    print('파일 형식 변환 완료...!')
    emotion_audio = emotion.load_audio(audio_path)
    emotion_mels_audio, emotion_mfcc_audio = emotion.mels_mfcc_array(emotion_audio)
    emotion_pred = emotion.predict(emotion_mels_audio, emotion_mfcc_audio)[0]

    if emotion_pred == 0:
        emo = '기쁨'
    elif emotion_pred == 1:
        emo = '슬픔'
    elif emotion_pred == 2:
        emo = '분노'
    elif emotion_pred == 3:
        emo = '당황'
    elif emotion_pred == 4:
        emo = '중립'
    
    print()
    print('------------------------------------------------')
    print("감정: ", emo)
    print('------------------------------------------------')
    print()
    return emo, audio_path


In [13]:
def return_answer_by_chatbot(user_text, emo_model):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = emo_model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  print("시스템 응답: ",chatbot_response, '\n')
  return chatbot_response

In [22]:
def todagtodag(person_audio):
    # 음성 감정 분석
    emotion, audio_path = emotion_predict(person_audio)

    # 음성을 텍스트로 변환
    r = sr.Recognizer()

    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
    try:
        stt = r.recognize_google(audio_data=audio, language='ko-KR')
        print("사용자: ",stt)
    except sr.UnknownValueError:
        print("다시 녹음해주세요!")
        record()
    except sr.ValueError:
        print("다시 녹음해주세요!")
        record()
        


    # Cht Bot
    if emotion == '기쁨':
        chat = return_answer_by_chatbot(stt, ha_model)
    elif emotion == '슬픔':
        chat = return_answer_by_chatbot(stt, sa_model)
    elif emotion == '분노':
        chat = return_answer_by_chatbot(stt, an_model)
    elif emotion == '당황':
        chat = return_answer_by_chatbot(stt, em_model)
    elif emotion == '중립':
        chat = return_answer_by_chatbot(stt, ne_model)


    # 텍스트를 음성으로 변환
    an = audio_path[audio_path.rfind('/')+1:][:-4]
    system_answer = gTTS(chat, lang='ko')
    system_answer.save(CP2_PATH+f'System_ans/{an}_chatbot.wav')

    print("시스템 답변 생성중... \n")
    sleep(20)
    print("시스템 답변 생성 완료!")
    display(Audio(CP2_PATH+f'System_ans/{an}_chatbot.wav', autoplay=True))    

In [20]:
import tempfile
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
import io
from scipy.io.wavfile import read, write

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
    print("Speak Now...")
    display(Javascript(RECORD))
    sec += 1
    s = output.eval_js('record(%d)' % (sec*1000))
    print("Done Recording !")
    b = b64decode(s.split(',')[1])

    tmp_path = tempfile.mkdtemp()
    with open(tmp_path+'/person_audio.m4a', mode = 'bx') as f:
            f.write(b)

    todagtodag(tmp_path+'/person_audio.m4a')

    #os.rmdir(tmp_path)

    

In [18]:
record()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
파일 형식 변환 중....
파일 형식 변환 완료...!
audio mels shape:  (1, 120, 450, 1)
audio mfcc shape:  (1, 120, 450, 1)
[[0.00352698 0.21632874 0.16255613 0.01898465 1.5986035 ]]

------------------------------------------------
감정:  중립
------------------------------------------------

사용자:  나는 지금 기분이 너무너무 좋지가 않아
시스템 응답:  지금 기분이 조금은 이해 되네요 

시스템 답변 생성중... 

시스템 답변 생성 완료!


In [19]:
record()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
파일 형식 변환 중....
파일 형식 변환 완료...!
audio mels shape:  (1, 120, 450, 1)
audio mfcc shape:  (1, 120, 450, 1)
[[2.5680833e-05 9.2692912e-02 1.0731666e+00 8.3183408e-01 2.2808767e-03]]

------------------------------------------------
감정:  분노
------------------------------------------------

사용자:  나 감기에 걸렸어 기침 나 안양 그거 나한테 옮겼어
시스템 응답:  감기에 걸리셨군요 

시스템 답변 생성중... 

시스템 답변 생성 완료!


In [21]:
record()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
파일 형식 변환 중....
파일 형식 변환 완료...!
audio mels shape:  (1, 120, 450, 1)
audio mfcc shape:  (1, 120, 450, 1)
[[8.0441343e-05 9.2107379e-01 7.8077324e-02 1.0000149e+00 7.5358129e-04]]

------------------------------------------------
감정:  당황
------------------------------------------------

사용자:  이날 취업했어 나 네이버가
시스템 응답:  취업하셨다니 정말 다행이에요. 

시스템 답변 생성중... 

시스템 답변 생성 완료!


In [23]:
record()

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
파일 형식 변환 중....
파일 형식 변환 완료...!
audio mels shape:  (1, 120, 450, 1)
audio mfcc shape:  (1, 120, 450, 1)
[[0.00135328 0.01135161 0.7761847  1.2005017  0.01060884]]

------------------------------------------------
감정:  당황
------------------------------------------------

사용자:  나 토스에 합격했을때
시스템 응답:  토스는 토스에합격할 때 가장 축하 해요! 

시스템 답변 생성중... 

시스템 답변 생성 완료!
